## Prim's Algorithm for Minimum Spanning Tree

The minimum spanning tree problem is about connecting a bunch of objects as cheaply as possible. The objects and connections could represent something physical, like computer servers and communication links between them. A graph $G=(V,E)$ has two ingredients: a set V of vertices and a set E of edges. We consider undirected graphs in which an edge e is an unordered pair {v,w} of vertices, which are called the endpoints of the edge. The number of vertices |V| and number of edges |E| are denoted n and m respectively. 

**Minimum Spanning Tree (MST)**

**Input:** A connected undirected graph G=(V,E) and a real-values cost $c_e$ for each edge $e\in E$.

**Output:** A spanning tree $T\subseteq E$ of G with minimum possible sum $\sum_{e\in T} c_e$ of edge costs.


**Prim's Algorithm**

**Input:** A connected undirected graph $G=(V,E)$ in adjacency-list representation and a cost $c_e$ for each edge $e\in E$.

**Output:** The edges of a minimum spanning tree of G.

------------------------------------

// Initialization

$X := {s}$   // choosing arbitary starting vertex s

$T := \empty$ // T = edges that span X

**while** there is $(v,w)$ with $v\in X, w\notin X$ **do**

    $(v*,w*)$ := a minimum-cost such edge
    
    add vertex $w*$ to X
    
    add edge $(v*,w*)$ to T

return T

-------------------------------------------

The `edges.txt` file describes an undirected graph with integer edge costs.  It has the format

[number_of_nodes] [number_of_edges]

[one_node_of_edge_1] [other_node_of_edge_1] [edge_1_cost]

[one_node_of_edge_2] [other_node_of_edge_2] [edge_2_cost]

...

For example, the third line of the file is "2 3 -8874", indicating that there is an edge connecting vertex #2 and vertex #3 that has cost -8874. 

You should NOT assume that edge costs are positive, nor should you assume that they are distinct.

Your task is to run Prim's minimum spanning tree algorithm on this graph.  You should report the overall cost of a minimum spanning tree --- an integer, which may or may not be negative

**Min-Heap Implementation**

In [72]:
class MinHeap:

    def __init__(self, maxSize):
        self.maxSize = maxSize
        self.arr = [None]*maxSize
        self.heapSize = 0

    def heapify(self, i):
        l = 2*i + 1
        r = 2*i + 2
        minimum = i
        if l < self.heapSize and self.arr[l].key < self.arr[i].key:
            minimum = l
        if r < self.heapSize and self.arr[r].key < self.arr[minimum].key:
            minimum = r
        if minimum != i:
            tmp = self.arr[i]
            self.arr[i] = self.arr[minimum]
            self.arr[minimum] = tmp
            self.heapify(minimum)

    def insert(self, x):
        if self.heapSize == self.maxSize:
            print("Overflow: Cannot insert any more node.")
            return
        
        self.heapSize += 1
        i = self.heapSize - 1
        self.arr[i] = x
        parent = (i - 1)//2
        while i != 0 and self.arr[parent].key > self.arr[i].key:
            tmp = self.arr[i]
            self.arr[i] = self.arr[parent]
            self.arr[parent] = tmp
            i = parent
            parent = (i - 1)//2

    def removeMin(self):
        if self.heapSize <= 0:
            return None
        if self.heapSize == 1:
            self.heapSize = 0
            minimum = self.arr[0]
            self.arr[0] = None
            return minimum
        
        minimum = self.arr[0]
        self.arr[0] = self.arr[self.heapSize - 1]
        self.arr[self.heapSize - 1] = None
        self.heapSize -= 1
        self.heapify(0)
        return minimum
    
    # Increases value of key at
    # index 'i' to new_val.
    def increaseKey(self, i, newVal):
        self.arr[i].key = newVal
        parent = (i - 1)//2
        while i != 0 and self.arr[parent].key > self.arr[i].key:
            tmp = self.arr[i]
            self.arr[i] = self.arr[parent]
            self.arr[parent] = tmp
            i = parent
            parent = (i - 1)//2
    
    def deleteKey(self, i):
        # It increases the value of the key
        # to -infinity and then removes
        # the minimum value.
        self.increaseKey(i, -float("inf"))
        self.removeMin()

    def updateHeap(self, w):
        # update l_min for each node connected to w
        # print("Before update d")
        # self.printHeap()
        for i in range(self.heapSize):
            if w.id in self.arr[i].edges:
                # print("Node {} has incoimng edge from node {}".format(self.arr[i].id, w.id))
                v = self.arr[i]
                tmp_key = v.key
                self.deleteKey(i)
                # print("Delete {}".format(v.id))
                # self.printHeap()
                v.key = tmp_key
                if v.key > v.edges[w.id]:
                     v.key = v.edges[w.id]
                     v.winner = (w.id, v.id, v.key) 
                   
                self.insert(v)
                # print("Re-insert {}".format(v.id))
                # self.printHeap()
        # print("After update d")
        # self.printHeap()
        
        return

    def getMin(self):
        return self.arr[0]
    
    def printHeap(self):
        heap = ''
        for i in range(self.heapSize):
            heap += "{}:{}, ".format(self.arr[i].id, self.arr[i].key)
        print(heap)

**Vertice Class**

In [73]:
class Vertice:

    def __init__(self, id):
        self.id = id        # label of the vertice
        self.key = float("inf") # key value 
        self.winner = None
        self.edges = {}     # store of connected edge i.e., vertice: cost
         

    def __str__(self):
        return str(self.id)

In [74]:
def prim_heap(G):
    # Select the vertex 1 as starting vertex
    s = G[1]
    # initialize T to empty
    T = []
    # initialize H
    H = MinHeap(len(G))
    # Initialize X with s
    X = [s]
    VX = [v for v in G.values()]
    VX.remove(s)

    for v in VX:
        if v.id in s.edges:
            v.key = s.edges[v.id]
            v.winner = (s.id, v.id, v.key)
        H.insert(v)
    
    while VX:
        # H.printHeap()
        w = H.removeMin()
        # H.printHeap()
        H.updateHeap(w)
        
        # if w not in VX:
        # H.printHeap()
        # print(len(X))
        # print(len(VX))

        X.append(w)
        T.append(w.winner)
        VX.remove(w)
        
        # for y in VX:
        #     if w.id in y.edges:
        #         c_wy = y.edges[w.id]
        #         if c_wy < y.key:
        #             H.deleteKey(y.id)
        #             y.key = c_wy
        #             y.winner = (w.id, y.id, y.key)
        #             H.insert(y)
    return T

In [84]:
def prim(G, nodes):
    """ implement Prim's algorithm on MST """
    s = G[1]
    T = []
    X = [s]
    V = [v for v in G.values()]
    span = {list(nodes)[0]}
    cost = []
    while span != nodes:
        min_cost = float("inf")
        for v1 in span:
            for v2 in nodes - span:
                if v1 in G[v2].edges and G[v2].edges[v1] < min_cost:
                    node = v2
                    span_edge = (v1, v2, G[v2].edges[v1])
                    min_cost = G[v2].edges[v1]

                if v2 in G[v1].edges and G[v1].edges[v2] < min_cost:
                    node = v2
                    span_edge = (v1, v2, G[v1].edges[v2])
                    min_cost = G[v1].edges[v2]

                # if (v2, v1) in graph and graph[(v2, v1)] < min_cost:
                #     node = v2
                #     min_cost = graph[(v2, v1)]
        # X.append(node)
        # V.remove(node)
        T.append(span_edge)
        span.add(node)
    return T

In [85]:
with open('edges.txt') as f:
    lines = f.readlines()

# Initialize Graph
n = int(lines[0].strip().split()[0])
G = {i:Vertice(i) for i in range(1,n+1)}
nodes = set()
for line in lines[1:]:
    data = line.rstrip().split()
    v = int(data[0])
    w = int(data[1])
    c = int(data[2])
    G[v].edges[w] = c
    # G[w].edges[v] = c
    nodes.add(v)
    nodes.add(w)

# for i, v in G.items():
#     print("{}:{}".format(i, v.edges))
T = prim(G, nodes)
spanning_cost = 0
for e in T:
    spanning_cost += e[2]
spanning_cost

-3612829